# Comunicação ponto-a-ponto confiável com detecção e correção de erros

## Introdução

O objetivo desse trabalho é implementar um protocolo confiável para comunicação entre dois computadores usando a linguagem Python. Para demonstrar o funcionamento do código, as células desse notebook devem ser executadas na ordem em que aparecem e dois jupyter-notebooks devem ser abertos, um para ser o servidor e outro para ser o cliente. <br>
O protocolo implementado emprega a transmissão *stop-and-wait* onde o emissor envia um quadro e aguarda a confirmação do receptor antes de enviar o próximo. Para a detecção de erros foi implementado o método *Cyclic Redundancy Check* (CRC), ou, em tradução livre, Verificação Cíclica de Redundância.


## Detecção de Erros 

Para a detecção de erros foi usado o método *Cyclic Redundancy Check* (CRC), ou, em tradução livre, Verificação Cíclica de Redundância com o polinômio: 
$$x^{16} + x^{15} + x^{2} + 1$$
Para facilitar sua posterior modificação, o polinômio em sua forma binária foi instanciado como uma constante:

In [1]:
POLINOMIO = "11000000000000101"

Foi criada uma classe para controle do CRC que tem como atributos o polinômio e seu grau. Os métodos dessa classe são basicamente dois:
<ul>
    <li>**geraCRC()**, que recebe uma sequência de bits correspondente à mensagem com cabeçalho e retorna a sequência de bits de verificação que devem ser colocados no final do quadro</li>
    <li>**verificaCRC()**, que faz o inverso do primeiro método, recebe um quadro já com os bits de verificação e realiza verificação, retornando uma variável booleana de acordo com o resultado da verificação</li>
</ul>
A entrada dessas funções deve ser feita no formato que o Python trata números binários, que é representado por '0b' seguido dos bits. O calculo realizado na geração do CRC é composto dos seguintes passos:
<ol>
    <li>Concatenar a quantidade de '0' correspondente ao grau do polinômio. Por exemplo, um polinômio '1001' exigiria que fosse concatenado três '0' ao final do quadro antes de começar o cálculo</li>
    <li>Realizar o XOR bit-a-bit do resultado do primeiro passo com o polinômio gerador</li>
</ol>
O resultado dessa sequência de passos é o código CRC. Note que o código CRC deve ter a mesma quantidade de bits que o grau do polinômio, portanto são aceitos zeros à esquerda para preenchimento. Por exemplo, para um polinômio '1001' a função deve retornar um código CRC de 3 bits, já que o grau do polinômio é 3. <br>
Já para realizar a verificação do CRC só é necessário passar o quadro com o código CRC no final e realizar o XOR bit-a-bit novamente e então verificar se o novo código gerado é composto apenas por zeros. <br>
Como essas duas funções compartilhavam o calcúlo do XOR com o polinômio gerador, uma terceira função foi criada para realizar esse cálculo, essa função é a **calculaCRC()**, que recebe uma lista de bits e retorna o resultado do XOR desses bits com o polinomio gerador.


In [2]:
class CRC:
    polinomio = None
    polinomioDecimal = None
    grauPolinomio = None
    
    def __init__(self, polinomio = POLINOMIO):
        self.polinomio = polinomio
        self.polinomioDecimal = int(self.polinomio, base=2)
        self.grauPolinomio = len(self.polinomio) - 1
        
    # Entrada: mensagem com bits de verificação inclusos (exemplo: "0b1110111101")
    # Retorna True se o resultado do CRC for 0
    def verificaCRC(self, mensagem):
        print(mensagem)
        mensagem = list(mensagem[2:])
        resultado = "0b" + ''.join(self.calculaCRC(mensagem))
        if(int(resultado, base = 2) == 0):
            return True
        return False

    # Entrada: um valor binario (exemplo: '0b110100101001')
    # Retorna: um binario com a sequencia de bits gerada pelo calculo do CRC (exemplo: '0b01001')
    def geraCRC(self, mensagem):
        mensagem = list(mensagem[2:] + ('0' * self.grauPolinomio))
        retorno = "0b" + ''.join(self.calculaCRC(mensagem))
        return retorno

    # Entrada: recebe uma lista de caracteres com valores de 1 e 0 (exemplo: ['1', '0', '1'])
    # Retorna: lista com caracteres de valores 1 ou 0 (exemplo: ['1', '0', '1'])
    def calculaCRC(self, mensagem):
        for i in range(len(mensagem) - self.grauPolinomio):
            if(mensagem[i] == '1'):
                for j in range(len(self.polinomio)):
                    bitMensagem = int(mensagem[i + j])
                    bitPolinomio = int(self.polinomio[j])
                    mensagem[i + j] = str(bitMensagem ^ bitPolinomio)

        mensagem = mensagem[-self.grauPolinomio:]
        return mensagem

Para exemplificarmos o funcionamento dessa classe, tomaremos a mensagem "1101011011" e o polinômio $x^{4} + x + 1$ em sua forma binária "10011". Primeiramente instanciaremos um objeto da classe CRC passando o polinômio como argumento.

In [4]:
testeCRC = CRC("10011")

Então, podemos gerar o código CRC:

In [5]:
codigoCRC = testeCRC.geraCRC("1101011011")
print(codigoCRC)

0b1100


E com esse resultado, podemos verificar o CRC. A função retornará True se o resultado do XOR bit-a-bit der resultado 0.

In [7]:
print(testeCRC.verificaCRC("1101011011" + codigoCRC[2:]))

11010110111100
True


Para comprovar o funcionamento da função podemos alterar um bit da mensagem, simulando um erro na transferência do quadro. Observe que o terceiro bit da mensagem anterior foi alterado de 0 para 1:

In [8]:
print(testeCRC.verificaCRC("1111011011" + codigoCRC[2:]))

11110110111100
False
